In [ ]:
import numpy as np

class Prism:
    def __init__(self):
        self.rules = []

    def fit(self, X, y):
        self.classes = np.unique(y)
        data = np.column_stack((X, y))

        for class_i in self.classes:
            class_i_data = data.copy()

            while class_i_data[class_i_data[:, -1] == class_i].size > 0:
                rule = self._build_rule(class_i_data, class_i)
                self.rules.append((rule, class_i))

                covered_indices = self._get_covered_indices(rule, class_i_data)
                class_i_data = np.delete(class_i_data, covered_indices, axis=0)

    def _build_rule(self, data, class_i):
        rule = []
        subset = data.copy()

        while np.unique(subset[:, -1]).size > 1:
            best_prob = -1
            best_subset = None
            best_attr_val = None

            for attr in range(data.shape[1] - 1):
                unique_vals = np.unique(data[:, attr])
                for val in unique_vals:
                    current_subset = data[data[:, attr] == val]
                    prob = np.sum(current_subset[:, -1] == class_i) / current_subset.shape[0]

                    if prob > best_prob:
                        best_prob = prob
                        best_subset = current_subset
                        best_attr_val = (attr, val)

            subset = best_subset
            rule.append(best_attr_val)

        return rule

    def _get_covered_indices(self, rule, data):
        indices = []
        for i, instance in enumerate(data):
            if all([instance[attr] == val for attr, val in rule]):
                indices.append(i)
        return indices

    def predict(self, X):
        predictions = []
        for instance in X:
            predicted_class = None
            for rule, class_i in self.rules:
                if all([instance[attr] == val for attr, val in rule]):
                    predicted_class = class_i
                    break
            predictions.append(predicted_class)
        return np.array(predictions)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

In [ ]:
X = np.array([[1, 1], [2, 1], [1, 2], [2, 2], [3, 3], [4, 4], [4, 3], [3, 4]])
y = np.array([0, 0, 0, 0, 1, 1, 1, 1])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

prism = Prism()
prism.fit(X_train, y_train)

predictions = prism.predict(X_test)
accuracy = prism.score(X_test, y_test)
print(f"Acurácia: {accuracy * 100:.2f}%")